# CSV 實例計算關聯規則

讀取資料。

In [1]:
import pandas as pd

store_data = pd.read_csv('retail_dataset.csv')
store_data

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
310,Bread,Eggs,Cheese,NaN,NaN,NaN,NaN
311,Meat,Milk,Pencil,NaN,NaN,NaN,NaN
312,Bread,Cheese,Eggs,Meat,Pencil,Diaper,Wine
313,Meat,Cheese,NaN,NaN,NaN,NaN,NaN


建立清理後的資料集。做法具體是把每一列變成 Python 的列表，不含 NaN 值。

In [2]:
transactions = [
    [item for item in line if pd.notna(item)]
    for line in store_data.values
]

transactions[:3]

[['Bread', 'Wine', 'Eggs', 'Meat', 'Cheese', 'Pencil', 'Diaper'],
 ['Bread', 'Cheese', 'Meat', 'Diaper', 'Wine', 'Milk', 'Pencil'],
 ['Cheese', 'Meat', 'Eggs', 'Milk', 'Wine']]

將資料集進行編碼。

In [3]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_trans = pd.DataFrame(te_ary, columns=te.columns_)

df_trans

,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,False,True,True,True,True,True,False,True,True
1,False,True,True,True,False,True,True,True,True
2,False,False,True,False,True,True,True,False,True
3,False,False,True,False,True,True,True,False,True
4,False,False,False,False,False,True,False,True,True
...,...,...,...,...,...,...,...,...,...
310,False,True,True,False,True,False,False,False,False
311,False,False,False,False,False,True,True,True,False
312,False,True,True,True,True,True,False,True,True
313,False,False,True,False,False,True,False,False,False


這次我們使用 FP-Max 演算法找出高頻資料集。

> FP-Max is a variant of FP-Growth, which focuses on obtaining maximal itemsets. An itemset X is said to maximal if X is frequent and there exists no frequent super-pattern containing X. In other words, a frequent pattern X cannot be sub-pattern of larger frequent pattern to qualify for the definition maximal itemset.

In [4]:
from mlxtend.frequent_patterns import fpmax

frequent_itemsets = fpmax(df_trans, min_support=0.2, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.200000,"(Pencil, Bread)"
1,0.200000,"(Pencil, Cheese)"
2,0.200000,"(Pencil, Wine)"
3,0.200000,"(Cheese, Diaper)"
4,0.231746,"(Diaper, Bread)"
5,0.234921,"(Diaper, Wine)"
6,0.225397,"(Bagel, Milk)"
7,0.279365,"(Bagel, Bread)"
8,0.241270,"(Eggs, Wine)"
9,0.244444,"(Milk, Eggs)"


執行關聯規則。這裡以 `lift` 為指標，計算最小增益為 1.1 的規則。

In [5]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.25, support_only=True)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Bagel),(Bread),NaN,NaN,0.279365,NaN,NaN,NaN,NaN,NaN
1,(Bread),(Bagel),NaN,NaN,0.279365,NaN,NaN,NaN,NaN,NaN
2,(Meat),(Wine),NaN,NaN,0.250794,NaN,NaN,NaN,NaN,NaN
3,(Wine),(Meat),NaN,NaN,0.250794,NaN,NaN,NaN,NaN,NaN
4,(Cheese),(Wine),NaN,NaN,0.269841,NaN,NaN,NaN,NaN,NaN
5,(Wine),(Cheese),NaN,NaN,0.269841,NaN,NaN,NaN,NaN,NaN
6,(Milk),(Bread),NaN,NaN,0.279365,NaN,NaN,NaN,NaN,NaN
7,(Bread),(Milk),NaN,NaN,0.279365,NaN,NaN,NaN,NaN,NaN
